# 오픈소스 SW 활용 기말 프로젝트
#### 32202341 송지윤

#### cryptocurrency change in 2020-2022 COVID-19 

In [3]:
import pandas as pd
import requests

# API의 모든 코인 아이디, 이름 출력
# Define the API endpoint
url = "https://api.coingecko.com/api/v3/coins/list"

# Send a GET request to the API
response = requests.get(url)

# Parse the response and extract the coin data
data = response.json()

# Convert the coin data into a pandas DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


                             id symbol                       name
0                        01coin    zoc                     01coin
1                        0chain    zcn                        Zus
2                 0vix-protocol    vix              0VIX Protocol
3                            0x    zrx                0x Protocol
4      0x0-ai-ai-smart-contract    0x0  0x0.ai: AI Smart Contract
...                         ...    ...                        ...
10006                   zynergy    zyn                    Zynergy
10007                      zyro   zyro                       Zyro
10008                     zyrri    zyr                      Zyrri
10009                       zyx    zyx                        ZYX
10010                       zzz    zzz                GoSleep ZZZ

[10011 rows x 3 columns]


In [2]:
# 2020-2022 비트코인 가격 변화

# Define the API endpoint and parameters
url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range"
params = {
    "vs_currency": "usd",
    "from": 1577836800,  # UNIX timestamp for Jan 1, 2020
    "to": 1672444799,    # UNIX timestamp for Dec 31, 2022
}

# Send a GET request to the API
response = requests.get(url, params=params)

# Parse the response and extract the price data
data = response.json()
prices = data['prices']

# Convert the price data into a pandas DataFrame
df = pd.DataFrame(prices, columns=['Timestamp', 'Price'])

# Convert the timestamp to datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

# Print the DataFrame
print(df)


      Timestamp         Price
0    2020-01-01   7195.153895
1    2020-01-02   7193.754668
2    2020-01-03   6963.853354
3    2020-01-04   7300.346935
4    2020-01-05   7351.377053
...         ...           ...
1090 2022-12-26  16842.177040
1091 2022-12-27  16900.081872
1092 2022-12-28  16701.840211
1093 2022-12-29  16538.906094
1094 2022-12-30  16644.379087

[1095 rows x 2 columns]
